In [1]:
from classes import Neuron, Layer, Network_Model
import helpers as hlp
import pandas as pd
import math
import random as rnd
import numpy as np

In [2]:
mdl = Network_Model()
mdl.add_layer(3)
mdl.add_layer(1, activation_function='identity')

In [5]:
x = [0.1, 0.9, 0.1]
t = [1]

In [6]:
def get_error(): 
    y = mdl.predict(x)
    Err_p = 0
    E_p = []
    temp_sum = 0

    for j,y_j in enumerate(y):
        E_p.append((t[j] if type(t) == list else t) - y_j)

    temp_sum = 0
    for e in E_p:
        temp_sum += e * e

    Err_p = 1/2 * temp_sum
    
    print(x)
    print(y)
    print(t)
    print(E_p)
    print(Err_p)

    return(E_p)

In [7]:
E_p = get_error()

[0.1, 0.9, 0.1]
[-0.7612155271894685]
[1]
[1.7612155271894685]
1.5509400666066386


In [19]:
mdl.layers[1].get_weights()

,0
0,-0.440349
1,0.599707
2,0.891401
3,-0.003568


In [12]:
delta_w = []
learning_rate = 0.5

for layer in range(len(mdl.layers) - 1, 0, -1):
    is_output_layer = layer == len(mdl.layers) - 1

    # current layer neurons
    neurons_h = mdl.layers[layer].neurons

    # previous layer neurons
    neurons_k = mdl.layers[layer - 1].neurons.copy()
    neurons_k.insert(0, mdl.layers[layer].bias_neuron)

    # following layer neurons
    neurons_l = None if is_output_layer else mdl.layers[layer + 1].neurons
    
    act_func = mdl.layers[layer].activation_function
    delta_w.insert(0,[])

    for j,h in enumerate(neurons_h):
        delta_w[0].append([])
        act_der = h.do_activate_der(act_func)
        for i,k in enumerate(neurons_k):
            del_h = 0

            # del_h for output neurons
            if is_output_layer:
                del_h = act_der * E_p[h.id[1] - 1]
                h.set_delta(del_h)

            # del_h for hidden neurons
            else:
                for l in neurons_l:
                    del_h += l.delta * mdl.layers[layer + 1].weights[h.id[1]][l.id[1] - 1]
                del_h *= act_der
                h.set_delta(del_h)

            w = learning_rate * k.output * del_h
            delta_w[0][-1].append(w)

delta_w_t = list(map(list, zip(*delta_w)))

In [9]:
for i,l in enumerate(mdl.layers):
    if i != 0:
        print('Layer ' + str(i))
        print('')
        print('current weights ')
        print(l.get_weights())
        print('')
        print('delta weights ')
        delta_i_t = list(map(list, zip(*delta_w[i - 1])))
        print(pd.DataFrame(delta_i_t))

    print('')


Layer 1

current weights 
          0
0 -0.775944
1 -0.900497
2  0.099626
3  0.151148

delta weights 
          0
0  0.880608
1  0.088061
2  0.792547
3  0.088061



In [13]:
for i,l in enumerate(mdl.layers):
    if i != 0:
        delta_i_t = list(map(list, zip(*delta_w[i - 1])))
        for k in range(len(l.weights)):
            for h in range(len(l.weights[0])):
                print('changed weight w_' + str(i) + '_' + str(k) + '_' + str(h) + ' from ' + str(l.weights[k][h]) + ' to ' + str(l.weights[k][h] + delta_i_t[k][h]))
                l.weights[k][h] = l.weights[k][h] + delta_i_t[k][h]

changed weight w_1_0_0 from 0.10466353970934583 to 0.1795151996148982
changed weight w_1_1_0 from -0.8124361226280852 to -0.8049509566375299
changed weight w_1_2_0 from 0.892173219685148 to 0.9595397136001451
changed weight w_1_3_0 from 0.23920855025724722 to 0.24669371624780245


In [14]:
E_p = get_error()

[0.1, 0.9, 0.1]
[0.9872752178160561]
[1]
[0.012724782183943906]
8.096004081440811e-05
